# Predicting the possibility of heart disease with logistic regression

### Importing data and dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, precision_recall_curve
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
import lightgbm as lgb


In [ ]:
df = pd.read_csv('framingham.csv')
df.head()

### Inspecting Data

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
corr_matrix = df.corr()

# Plot the correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
df.drop_duplicates(inplace=True)
df.isna().sum()

In [7]:
# Dropping rows with more than 3 missing values
df.dropna(thresh=len(df.columns) - 3, inplace=True)

In [ ]:
df.isna().sum()

Dropping rows with several missing entries did not change anything on our dataframe. This means there was no row with such parameters.

In [ ]:
sns.heatmap(df.isna(), cbar=True)

### Handling missing data

In [ ]:
# Selecting columns with missing values
columns_with_na = df.columns[df.isna().any()].tolist()

# Describing the distribution of columns with missing values
for col in columns_with_na:
    print(f"Column: {col}")
    print(f"Mean: {df[col].mean()}")
    print(f"Median: {df[col].median()}")
    print(f"Mode: {df[col].mode()[0]}")
    print(f"Standard Deviation: {df[col].std()}")
    print("\n")
    
    # Plot histogram
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(df[col], kde=True, color='blue')
    plt.title(f'Histogram of {col}')
    
    # Plot boxplot
    plt.subplot(1, 2, 2)
    sns.boxplot(y=df[col], color='lightblue')
    plt.title(f'Boxplot of {col}')
    
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Create a larger figure
plt.figure(figsize=(12, 8))

# Create the boxplot
boxplot = df.boxplot(grid=False, rot=45, fontsize=10, patch_artist=True)

# Customize the boxplot
for box in boxplot.artists:
    box.set_facecolor('lightblue')

# Add title and labels
plt.title('Boxplot of DataFrame Columns', fontsize=15)
plt.xlabel('Columns', fontsize=12)
plt.ylabel('Values', fontsize=12)

# Show the plot
plt.show()

It is clear that none of the columns with missing values are normally distributed. Therefore, we will use the median to fill in the missing values, where applicable.

#### Exploring connections between variables

In [ ]:
# Exploring the connection between sex and the columns with missing values
for col in columns_with_na:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x=col, hue='male', bins=10, kde=True)
    plt.title(f'{col} Distribution by Sex')
    plt.show()

In [ ]:
# including age groups into the model
df2 = df.copy()
df2['age_group'] = pd.cut(df['age'], bins=[20, 30, 40, 50, 60], labels=['20-30', '30-40', '40-50', '50-60'])
for col in columns_with_na:
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df2, x='age_group', y=col, hue='male')
    plt.title(f'{col} Distribution by Age Group and Sex')
    plt.show()

Based on the plots, heartRate is mostly influenced by the sex of the person, while BMI, totChol, cigsPerDay and education are affected by both age and sex. Based on this and their distributions, I imputed the mean or median value of the respective category for each instance with missing values.

#### Imputing missing values

In [14]:
mean_heartrate_by_sex = df2.groupby('male')['heartRate'].mean()

#Defining a function to fill missing values based on the age
def impute_heartrate(row):
    if pd.isnull(row['heartRate']):
        # If heartrate is NaN, fill with mean heartrate of that age
        return mean_heartrate_by_sex[row['male']]
    else:
        return row['heartRate']

# Apply the function to the DataFrame
df2['heartRate'] = df2.apply(impute_heartrate, axis=1)

In [ ]:
#Same method used as above, but implenting a loop for all columns that are affected by both age and sex
affected_columns = ['BMI', 'totChol', 'cigsPerDay']

for col in affected_columns:
    # Calculate median values for each age_group - sex combination and overall as well
    median_col_by_age_sex = df2.groupby(['age_group', 'male'])[col].median()
    overall_median = df2[col].median()
    def impute_median(row, col=col):
        if pd.isnull(row[col]):
            group_median = median_col_by_age_sex.get((row['age_group'], row['male']))
            if group_median is not None:  # Check if median exists
                return group_median
            else:
                return overall_median  # Use overall median as fallback
        else:
            return row[col]

    df2[col] = df2.apply(impute_median, axis=1)


In [ ]:
# The exact same method was used for education, however the fallback in this case is 0.
# The reason is that the age group 0-20 has no education, so the ordinal number 0 is used as a fallback.
mode_col_by_age_sex = df2.groupby(['age_group', 'male'])['education'].median()

def impute_mode(row):
    if pd.isnull(row['education']):
            group_median = median_col_by_age_sex.get((row['age_group'], row['male']))
            if group_median is not None:
                return group_median
            else:
                return 0
    else:
        return row['education']
    
df2['education'] = df2.apply(impute_mode, axis=1)

In [17]:
# BPMeds is a binary column, and based on the plots, it seems not to be affected by age and sex
# Therefore, we can use KNN imputer to fill the missing values, that maps the most likely data point based on the other columns
imputer = KNNImputer(n_neighbors=5)
df2[['BPMeds']] = imputer.fit_transform(df2[['BPMeds']])
# It takes the average of the 5 nearest neighbours, therefore I used round function to keep the binary distribution
df2['BPMeds'] = df2['BPMeds'].round().astype(int)

In [ ]:
df2.head()

In [19]:
df2.drop(["age_group"], axis=1, inplace=True)

In [ ]:
# Glucose did not show any correlation with age or sex either, but it contained a lot of missing values
# Therefore, I included every other variable to the model to predict the missing values
# The data is continuous, and the distribution is left skewed
# I tried log transformation to make the data more normal, and also implemented outlier capping to avoid the effect of outliers
# However the data was still not normal, and the distribution was not improved
df3 = df2.copy()

# Identifying outliers using IQR method
Q1 = df2['glucose'].quantile(0.25)
Q3 = df2['glucose'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Capping the outliers
df3['glucose'] = np.where(df2['glucose'] < lower_bound, lower_bound, df2['glucose'])
df3['glucose'] = np.where(df2['glucose'] > upper_bound, upper_bound, df2['glucose'])

# Log transformation
df3['target_log'] = np.log(df3['glucose'] + 1)

print(f"Column: target_log")
print(f"Mean: {df3['target_log'].mean()}")
print(f"Median: {df3['target_log'].median()}")
print(f"Mode: {df3['target_log'].mode()[0]}")
print(f"Standard Deviation: {df3['target_log'].std()}")
print("\n")

# Plotting histogram
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(df3['target_log'], kde=True, color='blue')
plt.title(f'Histogram of target_log')

# Plotting boxplot
plt.subplot(1, 2, 2)
sns.boxplot(y=df3['target_log'], color='lightblue')
plt.title(f'Boxplot of target_log')

plt.show()

In [21]:
# Due to the distribution and the number of outliers, I could not use linear regression to predict the missing values
# Therefore, I used Random Forest Regressor, which is more robust to outliers

# Splitting the data into training and testing sets
df_missing = df2[df2['glucose'].isnull()]
df_not_missing = df2[df2['glucose'].notnull()]

# X_train contains the predictor variables for the rows without missing values
X_train_imputation = df_not_missing.drop(['glucose'], axis=1)
y_train_imputation = df_not_missing['glucose']

# X_test contains the predictor variables for the rows with missing values
X_test_imputation = df_missing.drop(['glucose'], axis=1)

# training the model
# Define and fit the Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, max_depth=5)
model.fit(X_train_imputation, y_train_imputation)

# prediction
predicted_values = model.predict(X_test_imputation)

# imputation
df2.loc[df['glucose'].isnull(), 'glucose'] = predicted_values

In [ ]:
df2.isna().sum()

### Transforming the dataset for the logistic regression task

In [23]:
data = df2.drop(["TenYearCHD"], axis=1)
target = df2["TenYearCHD"]

In [24]:
scaler = StandardScaler()

cont_data = data.select_dtypes(include=['float64'])
int_data = data.select_dtypes(include=['int64'])

x_float = scaler.fit_transform(cont_data)
x = np.concatenate((x_float, int_data), axis=1)

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, target, test_size=0.2, random_state=1312)

### Model training and evaluation

In [ ]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy: .2f}")

In [ ]:
print(classification_report(y_test, y_pred))

Class imbalance heavily affected the recall rate for class 1 therefore the model is not effectively distinguishes whether someone has a chance of heart disease in the close future or not. To fix this, I try different sampling methods to fine-tune the model.

#### Model sampling

In [ ]:
# Undersampling the majority class first using RandomUnderSampler
rus = RandomUnderSampler(random_state=4)
x_us, y_us = rus.fit_resample(x_train, y_train)
y_pred_us = lr.predict(x_test)
print(classification_report(y_test, y_pred_us))

In [ ]:
# Using ensemble method on the undersampled data by the ratio of 7:1 to even out the training process
ensemble_model = EasyEnsembleClassifier(n_estimators=123, random_state=4)
ensemble_model.fit(x_us, y_us)
y_pred_ensemble = ensemble_model.predict(x_test)

print(classification_report(y_test, y_pred_ensemble))

In [ ]:
# Oversampling the minority class by using SMOTE on the oversampled data
smote = SMOTE(random_state=4)
x_os, y_os = smote.fit_resample(x_us, y_us)
lr.fit(x_os, y_os)
y_pred_os = lr.predict(x_test)
print(classification_report(y_test, y_pred_os))

In [ ]:
# Using class_weight='balanced' parameter in Logistic Regression on the resampled data
# This modifies the loss function to penalize the minority class more
lr_bal = LogisticRegression(class_weight='balanced')
lr_bal.fit(x_os, y_os)
y_pred_bal = lr_bal.predict(x_test)
print(classification_report(y_test, y_pred_bal))

In [ ]:
# Predicting probabilities and adjusting the threshold
y_prob = lr_bal.predict_proba(x_test)[:, 1]

threshold = 0.35
y_pred_adjusted = (y_prob >= threshold).astype(int)
print(classification_report(y_test, y_pred_adjusted))

# Plotting the Precision-Recall curve to find the optimal threshold for our problem
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)

plt.plot(thresholds, precision[:-1], label="Precision")
plt.plot(thresholds, recall[:-1], label="Recall")
plt.xlabel("Threshold")
plt.legend()
plt.show()

In [ ]:
# Computing ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plotting ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.show()

## Trying another model

Light Gradient Boosting Classifier is less sensitive to class imbalance therefore we might get a better result by using it as a classification model with the unbalance property set to True.

In [ ]:
lg = lgb.LGBMClassifier(is_unbalance=True, random_state=42)
lg.fit(x_train, y_train)
y_pred_lg = lg.predict(x_test)
print(classification_report(y_test, y_pred_lg))

Despite applying various sampling methods and balancing algorithms, class imbalance remained a significant issue in the heart disease prediction model. The precision of the positive class consistently failed to exceed 0.4, which, as shown in the precision-recall curve, represents the best balance point between these two metrics.

Given that our primary objective is to identify true positive cases of heart disease with the highest accuracy, maximizing recall for class 1 is paramount. This ensures that we catch as many actual cases of heart disease as possible, even if it means sacrificing some precision or overall accuracy. Therefore I have set the threshold based on the plot, to 0.35 to keep a reasonable amount of precision (and F1-Score) while having a good recall value. Missing true positive cases can have serious implications in this context, so prioritizing recall is critical, despite the presence of imbalanced data.

In a final attempt to address this challenge, I implemented a different classifier to compare its ability to manage imbalance. Unfortunately, the results were similar to those observed with logistic regression, indicating that the fundamental issue persists across different models. This suggests that more advanced techniques—such as cost-sensitive learning or further tuning of the decision threshold—may be required to enhance the model's performance in predicting heart disease.